In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)
print(SEED)

0


#### The Data

In [3]:
test_df = pd.read_csv('../../../anemia_ml4hc/data/test_set_constant.csv')
# test_df = pd.read_csv('../../../anemia_ml4hc/data/val_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [4]:
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 17), (14000,))

In [5]:
level=0.1

In [6]:
#missingness
if level==0.1:
    steps = int(12e6)
elif level==0.2:
    steps = int(10e6)
elif level==0.3:
    steps == int(12e6)
else:
    print('UNKNOWN LEVEL')
level, steps

(0.1, 12000000)

#### Testing

In [7]:
# dqn_model = utils.load_dqn3('../../../anemia_ml4hc/models/sb/dueling_double_dqn/dddqn_basic_3000000_steps.zip')
# steps = int(10e6)
# level=0.3
dqn_model = utils.load_dqn(#f'../../../anemia_ml4hc/models/sb/seed_{SEED}/dueling_ddqn_{steps}.pkl')
f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_ddqn_pr_noisiness_0.2_missingness_{level}_{steps}.pkl')
# dqn_model = utils.load_dqn3('../../final/models/dqn_noisy_6_missing_3_21300000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using just stable baselines (not 3)


Invalid MIT-MAGIC-COOKIE-1 key

ValueError: Error: the file ../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_0/dueling_ddqn_pr_noisiness_0.2_missingness_0.1_12000000.pkl could not be found

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
test_df[test_df.episode_length==test_df.episode_length.min()].iloc[-1]['trajectory']

In [ ]:
test_df.episode_length.min(), test_df.episode_length.max()

In [ ]:
# test_df[test_df.episode_length == test_df.episode_length.max()]

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

test_df.y_pred.value_counts()

#### Saving files

In [ ]:
# test_df.to_csv(f'../../../anemia_ml4hc/test_dfs/dqn_pr_test_df_15000000.csv', index=False)
# success_df.to_csv(f'../../lupus_trial/test_dfs/orig/dqn_success_df_9500000.csv', index=False)

In [ ]:
# for steps in [int(9.5e6), int(10e6), int(10.5e6), int(11e6), int(11.5e6), int(12e6), int(12.5e6), int(13e6), int(13.5e6), int(15e6), int(16e6), int(17e6), int(18e6), int(19e6), int(20e6), int(21e6), int(22e6), int(23e6), int(24e6)]:
#     training_env = utils.create_env(X_train, y_train)
#     dqn_model = utils.load_dqn3(f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_{steps}', 
#                                 training_env)
#     test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
#     success_rate, success_df = utils.success_rate(test_df)
#     avg_length, avg_return = utils.get_avg_length_reward(test_df)
#     print(f'steps: {steps}, success_rate: {success_rate}, avg_length: {avg_length}, avg_return: {avg_return}')

#### Confusion matrix and classification report

In [ ]:
# test_df = pd.read_csv('../../test_dfs/many_features/0.1/correlated/test_df3_missing3_12500000.csv')
# test_df.head()

In [ ]:
# utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
# utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# mylist = [ 95.486, 96.543, 97.257, 96.093]
# np.mean(mylist), np.std(mylist)